# This is going to be awesome

### Imports

In [82]:
import pandas as pd
import numpy as np

In [83]:
pre = pd.read_csv('data/pre.csv', quotechar='\t')
during = pd.read_csv('data/during.csv', quotechar='\t')
post = pd.read_csv('data/post.csv', quotechar='\t')
pre.columns

Index(['"Timestamp', '""Participant ID (top-right corner of the screen)""',
       '""How old are you?""', '""Gender?""',
       '""What describes you the best?""',
       '""How comfortable are you in expressing emotions?""',
       '""Have you ever played with the drummer bot dB before?""',
       '""How much prior experience do you have with interactive musical interfaces (e.g. sensor systems. digital control devices)?""',
       '""How often do you use AI tools for creativity (e.g. writing. coding. music. fine arts. food recipes. etc.)?""',
       '""Please indicate if you have any of the following health problems.""";'],
      dtype='object')

### Filter data

In [96]:
pre_columns = pre.columns
pre_data = pre[[pre_columns[1], pre_columns[3], pre_columns[4], pre_columns[5]]]
pre_data.columns = ['Participant_ID', 'Gender', 'Personal description', 'Comfortable with expressing emotions']
pre_data = pre_data.map(lambda x: x.strip('"') if isinstance(x, str) else x)
print(pre_data.head(), pre_data.shape)

  Participant_ID             Gender                Personal description  \
0           test  Prefer not to say  Student/semi-professional musician   
1             11  Prefer not to say  Student/semi-professional musician   
2      andsti-01               Male                 Avid music listener   
3      andsti-01               Male                 Avid music listener   
4         grup16             Female                 Avid music listener   

  Comfortable with expressing emotions  
0                                    3  
1                                    0  
2                                    2  
3                                    2  
4                                    4   (143, 4)


In [97]:
during_columns = during.columns
during_data = during[[during_columns[1], during_columns[2], during_columns[5]]]
during_data.columns = ['Session_ID', 'Participant_ID', 'Excitement']
during_data = during_data.map(lambda x: x.strip('"') if isinstance(x, str) else x)
print(during_data.head(), during_data.shape)

  Session_ID Participant_ID Excitement
0     ILLIAC      andsti-01          3
1       CYSP      andsti-01          1
2    VOYAGER      andsti-01          1
3      ALICE      andsti-01          1
4       CYSP         grup16          2 (434, 3)


In [98]:
post_columns = post.columns
post_data = post[[post_columns[1], post_columns[3], post_columns[7]]]
post_data.columns = ['Participant_ID', 'Control', 'Creative expression']
post_data = post_data.map(lambda x: x.strip('"') if isinstance(x, str) else x)
print(post_data.head(), post_data.shape)

  Participant_ID Control Creative expression
0      andsti-01       1                   2
1      andsti-02       1                   1
2      andsti-03       1                   2
3      andsti-04       1                   2
4      andsti-05       1                   3 (113, 3)
